In [1]:
from algorithm import Algorithm
from sage.crypto.sboxes import sboxes, monomial_function
from disjoint_set_union import RangeDSU_with_min
import copy
from collections import Counter

In [2]:
def ExhaustiveExtension(Alg, depth=1, LSx=(0,), LS=(0,), a=None, ddt2=None, cliques_above=None, cliques=None, cosets=None, LB=None):
    n = Alg.n
    m = Alg.m
    S = Alg.S
    global sols
    global leaves

    if a == None:
        sols = []
        leaves = Counter()
        for i in range(2**(n-1)):
            ExhaustiveExtension(Alg,a=i,LB=LB)
        return

    if depth == 1:
        cliques = RangeDSU_with_min(2**n)
        cliques_above = list(reversed(range(a+1,2**n)))
        ddt2 = Alg.compute_kddt(k=2)    
        if LB is not None:
            cosets = RangeDSU_with_min(2**n)

    if depth < n:
        a_max = 2**n-2**(n-1-depth)
    else:
        a_max = 2**n        

    while cliques_above:
        
        a_new = cliques_above.pop()
        
        if a_new >= a_max:
            break
            
        if (dx0 := a_new ^ a) in LSx:
            continue

        dy0 = S[a_new] ^ S[a]               
        dxdy0 = (dx0 << m) | dy0    
        
        cliques_new = copy.deepcopy(cliques)
        LS_new = set(LS)
        if LB is not None:
            cosets_new = copy.deepcopy(cosets)
        
        #merge cliques
        to_delete = set()
        for dxdy in LS:
            dxdy ^= dxdy0  
            LS_new.add(dxdy)
            for x1, x2 in ddt2[dxdy >> m][dxdy & (2**m-1)]:
                cliques_new.union(x1, x2)
                min1 = cliques.min(x1)
                min2 = cliques.min(x2)
                to_delete.add(max(min1,min2))

        cliques_above_new = [cl for cl in cliques_above if cl not in to_delete]

        if LB is not None:
            # merge cosets
            for coset in cosets_new.components():
                cosets_new.union(coset, coset ^ dx0)
            
            max_cl_new = {}
            for cl in cliques_above_new + [a]:
                sz = cliques_new.size(cl)
                coset = cosets_new.find(cl)
                max_cl_new[coset] = max(sz, max_cl_new.get(coset,0))
            sum_max = sum(max_cl_new.values())

            if sum_max < LB:
                leaves[depth] += 1
                continue
        
        LSx_new = set(LSx) | {v ^ dx0 for v in LSx}

        if depth == n:
            leaves[depth] += 1
            cl = cliques_new.ec(a_new)
            sols.append(tuple(cl))

        else:
            if LB is not None:
                ExhaustiveExtension(Alg, depth+1, LSx_new, LS_new, a_new, ddt2, cliques_above_new, cliques_new, cosets_new, LB)
            if LB is None:
                ExhaustiveExtension(Alg, depth+1, LSx_new, LS_new, a_new, ddt2, cliques_above_new, cliques_new, None, LB)

In [3]:
def evaluation(Alg,k):        # set k to an integer (reasonably at least n+2) or None
    ExhaustiveExtension(Alg,LB=k)
    distr = Counter(map(len,sols))
    if len(distr) != 0:
        print("vectorial linearity:", max(distr))
        print("number of affine approximations with at least", k, "common points:", len(sols))
        for a in sorted(distr.keys()):
            print("number of affine approximations with", a, "common points:", distr[a])
        print("number of leaves:", sum(leaves.values()))
        for l in sorted(leaves.keys()):
            print("number of leaves in depth ", l,": ",leaves[l],  sep='')
        print("one solution with", max(distr), "common points:", sols[0])
    else:
        print("vectorial linearity smaller than", k)

In [4]:
n = 5
S = sboxes['Ascon']
Alg = Algorithm(S,n,n)

In [5]:
evaluation(Alg,11)

vectorial linearity: 11
number of affine approximations with at least 11 common points: 32
number of affine approximations with 11 common points: 32
number of leaves: 8657
number of leaves in depth 1: 35
number of leaves in depth 2: 2251
number of leaves in depth 3: 4952
number of leaves in depth 4: 1169
number of leaves in depth 5: 250
one solution with 11 common points: (0, 1, 2, 4, 5, 12, 14, 17, 18, 21, 29)
